In [1]:
import pandas as pd 
import os

In [2]:
cur_dir = os.getcwd()

In [3]:
data_df = pd.read_csv(rf'{cur_dir}\data.csv')
columns = data_df.columns.tolist()

data_df = pd.melt(data_df,
                  id_vars = columns[:5],
                  var_name = 'TIME_PERIOD', 
                  value_name = 'OBS_VALUE')

In [4]:
print(f'\n1. Додано 2 нових колонки: TIME_PERIOD та OBS_VALUE\n\n{data_df.head()}\n')


1. Додано 2 нових колонки: TIME_PERIOD та OBS_VALUE

  FREQ.id  EDRPOU.id     WASTE.id  HAZARD.id MATERIAL.id TIME_PERIOD  \
0       A      32106  2112_1_1_04        104        07_6        2020   
1       A      32106  2112_2_9_01        104        07_2        2020   
2       A      32106  2210_3_1_05        104        07_2        2020   
3       A      32106  2220_1_3_02        104          08        2020   
4       A      32106  2416_1_1_02        104        07_2        2020   

   OBS_VALUE  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  



In [5]:
count_2021 = data_df.query("TIME_PERIOD == '2021'")['OBS_VALUE'].notna().sum()
count_2020 = data_df.query("TIME_PERIOD == '2020'")['OBS_VALUE'].notna().sum()
 
cng = round((count_2021 / count_2020) * 100, 2)

if cng > 100:
    print(f"2. Рівень заповненості звітності в 2021 році в порівнянні з 2020 роком виріс на {cng}%")
elif cng == 100:
    print(f"2. Рівень заповненості звітності в 2021 році в порівнянні з 2020 роком не змінився")
elif cng < 100:
    print(f"2. Рівень заповненості звітності в 2021 році в порівнянні з 2020 роком зменшився, і становить {cng}%")

2. Рівень заповненості звітності в 2021 році в порівнянні з 2020 роком виріс на 636.52%


In [6]:
edrpou_df = pd.read_csv(rf'{cur_dir}\edrpou.csv')

merged_df = data_df.merge(edrpou_df, how='left', left_on='EDRPOU.id', right_on='EDRPOU').drop(columns='EDRPOU')

print(f'\n3. Додано атрибути КВЕД та КАТОТТГ\n\n{merged_df.head()}\n')


3. Додано атрибути КВЕД та КАТОТТГ

  FREQ.id  EDRPOU.id     WASTE.id  HAZARD.id MATERIAL.id TIME_PERIOD  \
0       A      32106  2112_1_1_04        104        07_6        2020   
1       A      32106  2112_2_9_01        104        07_2        2020   
2       A      32106  2210_3_1_05        104        07_2        2020   
3       A      32106  2220_1_3_02        104          08        2020   
4       A      32106  2416_1_1_02        104        07_2        2020   

   OBS_VALUE              KATOTTG   KVED  
0        NaN  UA80000000000624772  64_11  
1        NaN  UA80000000000624772  64_11  
2        NaN  UA80000000000624772  64_11  
3        NaN  UA80000000000624772  64_11  
4        NaN  UA80000000000624772  64_11  



In [7]:
merged_df['OBL.id'] = merged_df['KATOTTG'].str[2:4]

grouped_df = merged_df.groupby(['HAZARD.id', 'MATERIAL.id', 'WASTE.id', 'TIME_PERIOD', 'OBL.id'])['OBS_VALUE'].sum().reset_index(name = 'OBS_VALUE')

print(f"\n4. Розраховано об'єм утворених відходів для кожного класу небезпеки, матеріалу відходу, виду відходу, року звітності на рівні області \n\n{grouped_df.head()}\n")


4. Розраховано об'єм утворених відходів для кожного класу небезпеки, матеріалу відходу, виду відходу, року звітності на рівні області 

   HAZARD.id MATERIAL.id     WASTE.id TIME_PERIOD OBL.id  OBS_VALUE
0        101        01_1  2820_2_9_01        2020     63        0.0
1        101        01_1  2820_2_9_01        2020     73        0.0
2        101        01_1  2820_2_9_01        2021     63        2.3
3        101        01_1  2820_2_9_01        2021     73        0.0
4        101        01_1  3210_3_1_01        2020     46        0.0



In [8]:
grouped_df = merged_df.groupby(['HAZARD.id', 'KVED', 'TIME_PERIOD', 'OBL.id'])['OBS_VALUE'].sum().reset_index(name = 'OBS_VALUE')

print(f"\n5. Розраховано об'єм утворених відходів для кожного класу небезпеки, виду економічної діяльності, року звітності на рівні області \n\n{grouped_df.head()}\n")


5. Розраховано об'єм утворених відходів для кожного класу небезпеки, виду економічної діяльності, року звітності на рівні області 

   HAZARD.id   KVED TIME_PERIOD OBL.id  OBS_VALUE
0        101  01_11        2020     05      0.259
1        101  01_11        2020     07      0.000
2        101  01_11        2020     12      0.377
3        101  01_11        2020     14      0.000
4        101  01_11        2020     18     13.557



In [10]:
grouped_df = merged_df.groupby(['TIME_PERIOD', 'OBL.id'])['OBS_VALUE'].sum().reset_index(name = 'OBS_VALUE')

grouped_df['OBS_VALUE'] = grouped_df['OBS_VALUE'].map(lambda x: f'{x:,.2f}')

print(f"\n6. Розраховано об'єм утворених відходів для кожного року звітності на рівні області \n\n{grouped_df.head()}\n")

grouped_df = merged_df.groupby('OBL.id')['OBS_VALUE'].sum().reset_index(name = 'OBS_VALUE')

grouped_df['Percentage'] = (grouped_df['OBS_VALUE'] / grouped_df['OBS_VALUE'].sum()) * 100

print(f"В регіоні {grouped_df.loc[grouped_df['Percentage'].idxmax(), 'OBL.id']} утворилаcь найбільша кількість відходів: {round(grouped_df['Percentage'].max(), 2)} %")


6. Розраховано об'єм утворених відходів для кожного року звітності на рівні області 

  TIME_PERIOD OBL.id      OBS_VALUE
0        2020     05     142,557.27
1        2020     07     183,769.54
2        2020     12  92,334,330.97
3        2020     14     249,237.80
4        2020     18      72,706.26

В регіоні 12 утворилаcь найбільша кількість відходів: 64.18 %


In [11]:
class_df = pd.read_json(rf'{cur_dir}\SSSU_CL_WASTE_HAZARD_CLASS(2.0.1).json')
class_df = pd.json_normalize(class_df['codes'], sep='_')

merged_df['HAZARD.id'] = merged_df['HAZARD.id'].astype(str).str.zfill(4)

merged_df = merged_df.merge(class_df, how = 'left', left_on = 'HAZARD.id', right_on = 'id').drop(columns=['id', 'annotations', 'names_en'])

print(f"\n7. Додано атрибути name, parent  \n\n{merged_df.head()}\n")



7. Додано атрибути name, parent  

  FREQ.id  EDRPOU.id     WASTE.id HAZARD.id MATERIAL.id TIME_PERIOD  \
0       A      32106  2112_1_1_04      0104        07_6        2020   
1       A      32106  2112_2_9_01      0104        07_2        2020   
2       A      32106  2210_3_1_05      0104        07_2        2020   
3       A      32106  2220_1_3_02      0104          08        2020   
4       A      32106  2416_1_1_02      0104        07_2        2020   

   OBS_VALUE              KATOTTG   KVED OBL.id  \
0        NaN  UA80000000000624772  64_11     80   
1        NaN  UA80000000000624772  64_11     80   
2        NaN  UA80000000000624772  64_11     80   
3        NaN  UA80000000000624772  64_11     80   
4        NaN  UA80000000000624772  64_11     80   

                               name parent  
0  IV клас - малонебезпечні відходи   0124  
1  IV клас - малонебезпечні відходи   0124  
2  IV клас - малонебезпечні відходи   0124  
3  IV клас - малонебезпечні відходи   0124  
4  IV

In [12]:
grouped_df = merged_df.groupby(['TIME_PERIOD', 'parent'])['OBS_VALUE'].sum().reset_index(name = 'OBS_VALUE')

total_df = grouped_df.merge(class_df, how = 'left', left_on = 'parent', right_on = 'id')
total_df = total_df.groupby(['TIME_PERIOD', 'parent_y'])['OBS_VALUE'].sum().reset_index(name = 'OBS_VALUE')
total_df = total_df.rename(columns={'parent_y': 'parent'})
result_df = pd.concat([grouped_df, total_df], ignore_index=True)

result_df['OBS_VALUE'] = result_df['OBS_VALUE'].map(lambda x: f'{x:,.2f}')

result_df = result_df.merge(class_df[['id', 'name']], how='left', left_on='parent', right_on='id')

print(f"\n8. Обраховано об'єм утворених відходів для груп класів небезпеки \n\n {result_df[['TIME_PERIOD', 'name', 'OBS_VALUE']]}")


8. Обраховано об'єм утворених відходів для груп класів небезпеки 

   TIME_PERIOD                       name       OBS_VALUE
0        2020  I-III. Небезпечні відходи       21,676.20
1        2020  IV. Не небезпечні відходи   97,120,975.66
2        2021  I-III. Небезпечні відходи      488,676.59
3        2021  IV. Не небезпечні відходи  341,571,689.62
4        2020     I-IV. Відходи в цілому   97,142,651.86
5        2021     I-IV. Відходи в цілому  342,060,366.21
